In [8]:
# Run this code cell to import relevant libraries

import numpy as np
import pandas as pd
import seaborn as sns

In [37]:
hotels = pd.read_csv("hotels.csv")
hotels = hotels.fillna(0)
hotels = hotels.drop(columns = ["company","reservation_status","reservation_status_date"])
print(hotels.head)

<bound method NDFrame.head of                hotel  is_canceled  lead_time  arrival_date_year  \
0       Resort Hotel            0        342               2015   
1       Resort Hotel            0        737               2015   
2       Resort Hotel            0          7               2015   
3       Resort Hotel            0         13               2015   
4       Resort Hotel            0         14               2015   
...              ...          ...        ...                ...   
119385    City Hotel            0         23               2017   
119386    City Hotel            0        102               2017   
119387    City Hotel            0         34               2017   
119388    City Hotel            0        109               2017   
119389    City Hotel            0        205               2017   

       arrival_date_month  arrival_date_week_number  \
0                    July                        27   
1                    July                        27   


In [38]:
from sklearn.preprocessing import OneHotEncoder

target = hotels["adr"]
tot = hotels.drop(columns = ["adr"])

categorical = OneHotEncoder().fit_transform(hotels[["hotel","arrival_date_month", "customer_type", "meal", "country", "market_segment","distribution_channel", "reserved_room_type","assigned_room_type", "deposit_type"]].values).toarray()
numerical = tot.drop(columns = ["hotel","arrival_date_month", "customer_type", "meal", "country", "market_segment","distribution_channel", "reserved_room_type","assigned_room_type", "deposit_type"]).values
data = np.append(categorical, numerical, axis=1)
print(data)

TypeError: Encoders require their input to be uniformly strings or numbers. Got ['int', 'str']

In [35]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_target, test_target = train_test_split(
    data, target, test_size=0.3, random_state=216)

In [36]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

linear_model = LinearRegression()
linear_model.fit(X=data, y=target)
predicted = linear_model.predict(data)

mse = mean_squared_error(target, predicted)
r2 = r2_score(target, predicted)
print(mse)
print(r2)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').